In [1]:
from utils.imports import *

Using TensorFlow backend.


In [2]:
def classifier(input_shape, width, kernel_size, pool_size):
    model = Sequential()

    model.add(Convolution3D(width*1, kernel_size[0], kernel_size[1], kernel_size[2],
                            border_mode='valid',
                            input_shape=input_shape))
    model.add(Activation('relu'))
    model.add(MaxPooling3D(pool_size=pool_size))
    model.add(Dropout(0.2))
    model.add(Convolution3D(width*2, kernel_size[0], kernel_size[1], kernel_size[2]))
    model.add(Activation('relu'))
    model.add(MaxPooling3D(pool_size=pool_size))
    model.add(Dropout(0.2))
    model.add(Convolution3D(width*4, kernel_size[0], kernel_size[1], kernel_size[2]))
    model.add(Activation('relu'))
    model.add(MaxPooling3D(pool_size=pool_size))
    model.add(Dropout(0.2))


    model.add(Flatten())
    model.add(Dense(512))
    model.add(Activation('relu'))
    model.add(Dropout(0.3))
    model.add(Dense(128))
    model.add(Activation('relu'))
    model.add(Dropout(0.4))
    model.add(Dense(2))
    model.add(Activation('softmax'))
    model.compile(loss='categorical_crossentropy',
          #optimizer=SGD(lr=1e-4, momentum=0.9, nesterov=True),
          optimizer=Adam(lr=1e-5),                  
          metrics=['accuracy'])
    return model

def get_net(input_shape, load_weight_path=None, features=False, mal=False):
    inputs = Input(shape=(1, 32, 32, 32), name="input_1")
    x = inputs
    x = AveragePooling3D(pool_size=(2, 1, 1), strides=(2, 1, 1), border_mode="same")(x)
    x = Convolution3D(64, 3, 3, 3, activation='relu', border_mode='same', name='conv1', subsample=(1, 1, 1))(x)
    x = MaxPooling3D(pool_size=(1, 2, 2), strides=(1, 2, 2), border_mode='valid', name='pool1')(x)

    # 2nd layer group
    x = Convolution3D(128, 3, 3, 3, activation='relu', border_mode='same', name='conv2', subsample=(1, 1, 1))(x)
    x = MaxPooling3D(pool_size=(2, 2, 2), strides=(2, 2, 2), border_mode='valid', name='pool2')(x)
    x = Dropout(p=0.3)(x)

    # 3rd layer group
    x = Convolution3D(256, 3, 3, 3, activation='relu', border_mode='same', name='conv3a', subsample=(1, 1, 1))(x)
    x = Convolution3D(256, 3, 3, 3, activation='relu', border_mode='same', name='conv3b', subsample=(1, 1, 1))(x)
    x = MaxPooling3D(pool_size=(2, 2, 2), strides=(2, 2, 2), border_mode='valid', name='pool3')(x)
    x = Dropout(p=0.4)(x)

    # 4th layer group
    x = Convolution3D(512, 3, 3, 3, activation='relu', border_mode='same', name='conv4a', subsample=(1, 1, 1))(x)
    x = Convolution3D(512, 3, 3, 3, activation='relu', border_mode='same', name='conv4b', subsample=(1, 1, 1),)(x)
    x = MaxPooling3D(pool_size=(2, 2, 2), strides=(2, 2, 2), border_mode='valid', name='pool4')(x)
    x = Dropout(p=0.5)(x)

    last64 = Convolution3D(64, 2, 2, 2, activation="relu", name="last_64")(x)
    out_class = Convolution3D(1, 1, 1, 1, activation="softmax", name="out_class_last")(last64)
    out_class = Flatten(name="out_class")(x)
    
    out_class = Dense(2)(out_class) 
    out_class = Activation('softmax')(out_class)
    
    model = Model(input=inputs, output=out_class)
    model.compile(optimizer=SGD(lr=1e-3, momentum=0.9, nesterov=True), loss='categorical_crossentropy', metrics=['accuracy'])
    #model.compile(optimizer=Adam(lr=1e-5), loss='categorical_crossentropy', metrics=['accuracy'])   
    return model

In [3]:
output_path = PATH['cls_train_cube_30']
output_true = PATH['cls_train_cube_30_true']
output_false = PATH['cls_train_cube_30_false']
model_paths = PATH['model_paths']
model_final = PATH['model_final']

In [4]:

def get_dirfiles(dir):
    file_list = []
    subset_path = os.listdir(dir)
    for _ in range(len(subset_path)):
        if subset_path[_] != '.DS_Store':
            file_list.append(dir + subset_path[_])
    return file_list

def train_generator(output_true,output_false):
    file_list_true = get_dirfiles(output_true)[0:6000]
    file_list_false = get_dirfiles(output_false)[0:6000]
    
    file_list_true = shuffle(file_list_true)
    file_list_false = shuffle(file_list_false)
    
    nb_true = len(file_list_true) + len(file_list_false)
    sample = np.zeros([nb_true,32, 32, 32])
    labels = np.zeros([nb_true,2])
    for i in tqdm(range(len(file_list_true))):       
        cc= np.load(file_list_true[i])
        sample[i] = cc
        labels[i] = [0.,1.]
    for j in tqdm(range(len(file_list_false))):
        bb= np.load(file_list_false[j])
        sample[j+len(file_list_true)] = bb 
        labels[j+len(file_list_true)] = [1.,0.]
    sample = np.expand_dims(sample, axis=1)        
    return sample,labels

def valid_generator(output_true,output_false):
    file_list_true = get_dirfiles(output_true)[-400:]
    file_list_false = get_dirfiles(output_false)[-400:]
    
    file_list_true = shuffle(file_list_true)
    file_list_false = shuffle(file_list_false)

    nb_true = len(file_list_true) + len(file_list_false)
    sample = np.zeros([nb_true,32, 32, 32])
    labels = np.zeros([nb_true,2])
  
    for i in tqdm(range(len(file_list_true))):       
        cc= np.load(file_list_true[i])
        sample[i] = cc
        labels[i] = [0.,1.]
    for j in tqdm(range(len(file_list_false))):
        bb= np.load(file_list_false[j])
        sample[j+len(file_list_true)] = bb 
        labels[j+len(file_list_true)] = [1.,0.]
    sample = np.expand_dims(sample, axis=1)        
    return sample,labels


def fenlei_fit(name, load_check = False,batch_size=2, epochs=100,check_name = None):

    t = time.time()
    callbacks = [EarlyStopping(monitor='val_loss', patience = 20, verbose = 1),
                 ModelCheckpoint((model_paths + '{}.h5').format(name),
                                 monitor='val_loss',
                                 verbose = 0,
                                 save_best_only = True)]
    if load_check:
        check_model = (model_paths + '{}.h5').format(check_name)
        model = load_model(check_model)
    else:
        #model = classifier((1, 32, 32, 32),128,(3, 3, 3), (2, 2, 2))
        model = get_net((1, 32, 32, 32))
    x,y = train_generator(output_true,output_false)
    model.fit(x=x, y=y, batch_size=batch_size, epochs=epochs,
              validation_data=valid_generator(output_true,output_false),verbose=1, callbacks=callbacks, shuffle=True)
    
 
    return model

In [5]:
fenlei_fit('Fenge_32_32_32_0625', load_check = False, batch_size=1, epochs=500, check_name = 'Fenge_32_32_32_0625')

100%|██████████| 400/400 [00:04<00:00, 94.21it/s] 


Train on 12000 samples, validate on 800 samples
Epoch 1/500
12000/12000 [==============================] - 545s - loss: 0.6935 - acc: 0.5293 - val_loss: 0.6830 - val_acc: 0.5737
Epoch 2/500
12000/12000 [==============================] - 543s - loss: 0.6031 - acc: 0.6724 - val_loss: 0.5379 - val_acc: 0.7338
Epoch 3/500
12000/12000 [==============================] - 542s - loss: 0.3697 - acc: 0.8373 - val_loss: 0.4890 - val_acc: 0.7937
Epoch 4/500
12000/12000 [==============================] - 535s - loss: 0.2451 - acc: 0.9002 - val_loss: 0.5953 - val_acc: 0.7700
Epoch 5/500
  171/12000 [..............................] - ETA: 538s - loss: 0.2729 - acc: 0.8889

KeyboardInterrupt: 

In [6]:
file_list_true = get_dirfiles(output_true)
file_list_false = get_dirfiles(output_false)

In [7]:
#model_pred = classifier((1, 36, 36, 36), (3, 3, 3), (2, 2, 2))
model_pred = load_model(model_paths + 'Fenge_32_32_32_0625.h5')

In [8]:
#model_pred.load_weights(model_paths + 'Fenge_0613_2.h5')

In [9]:
cc = []
for i in file_list_false[0:200]:
    a=np.load(i)
    a=np.expand_dims(a,0)
    a=np.expand_dims(a,0)
    cc.append(model_pred.predict(a))
count = 0
for i in cc:
    if i[0][0] > 0.7:
        count += 1
print count*1.0/len(cc)

0.86


In [10]:
cc

[array([[ 0.95530689,  0.04469314]], dtype=float32),
 array([[ 0.88495511,  0.11504491]], dtype=float32),
 array([[ 0.96871603,  0.03128394]], dtype=float32),
 array([[ 0.54430765,  0.45569232]], dtype=float32),
 array([[ 0.7244978 ,  0.27550223]], dtype=float32),
 array([[ 0.94870239,  0.05129759]], dtype=float32),
 array([[ 0.96343839,  0.03656165]], dtype=float32),
 array([[ 0.87322873,  0.12677132]], dtype=float32),
 array([[ 0.90534413,  0.09465592]], dtype=float32),
 array([[ 0.4375059,  0.5624941]], dtype=float32),
 array([[ 0.54653811,  0.45346186]], dtype=float32),
 array([[ 0.91641039,  0.08358964]], dtype=float32),
 array([[ 0.8726387 ,  0.12736125]], dtype=float32),
 array([[ 0.99745792,  0.0025421 ]], dtype=float32),
 array([[ 0.93017352,  0.06982645]], dtype=float32),
 array([[  9.99541283e-01,   4.58690512e-04]], dtype=float32),
 array([[ 0.99758995,  0.00241008]], dtype=float32),
 array([[  9.99157667e-01,   8.42378300e-04]], dtype=float32),
 array([[  9.99935746e-01,  

In [11]:
cc = []
for i in file_list_true[0:200]:
    a=np.load(i)
    a=np.expand_dims(a,0)
    a=np.expand_dims(a,0)
    cc.append(model_pred.predict(a))
count = 0


In [12]:
for i in cc:
    if i[0][1] > 0.7:
        count += 1
print count*1.0/len(cc)

0.825


In [13]:
cc

[array([[ 0.01559451,  0.98440546]], dtype=float32),
 array([[ 0.00228104,  0.99771899]], dtype=float32),
 array([[ 0.00604887,  0.9939512 ]], dtype=float32),
 array([[ 0.17558271,  0.82441723]], dtype=float32),
 array([[ 0.01291453,  0.98708552]], dtype=float32),
 array([[ 0.01093701,  0.98906296]], dtype=float32),
 array([[ 0.04000247,  0.95999753]], dtype=float32),
 array([[ 0.00903387,  0.9909662 ]], dtype=float32),
 array([[ 0.07721296,  0.92278707]], dtype=float32),
 array([[ 0.02324143,  0.9767586 ]], dtype=float32),
 array([[ 0.00487184,  0.99512815]], dtype=float32),
 array([[ 0.019209  ,  0.98079097]], dtype=float32),
 array([[ 0.13473544,  0.86526453]], dtype=float32),
 array([[ 0.17603926,  0.82396072]], dtype=float32),
 array([[ 0.10720459,  0.89279538]], dtype=float32),
 array([[ 0.02932145,  0.97067857]], dtype=float32),
 array([[ 0.01788201,  0.98211801]], dtype=float32),
 array([[ 0.05287098,  0.94712901]], dtype=float32),
 array([[ 0.01559451,  0.98440546]], dtype=flo

In [14]:
len(file_list_false)

6896